# Doing Data Science: chapter 2 - Housing Dataset

Python code for the exercise on the RealDirect study about the housing dataset ("*improve the way people sell and buy houses*").

**Author**: Damien Garaud

**Project on Github**: https://github.com/garaud/doing_pydata_science

## Getting Data 

Clone the official Github project at https://github.com/oreillymedia/doing_data_science and unzip the `dds_datasets.zip` file. You'll find a new ZIP file named `dds_chapter2_rollingsales.zip`. Then, you'll get five XLS files:

- `rollingsales_bronx.xls`
- `rollingsales_brooklyn.xls`
- `rollingsales_manhattan.xls`
- `rollingsales_queens.xls`
- `rollingsales_statenisland.xls`

**Note**: for each carried out task, I'll try to write a **function**. Functions are good.

## Modules

In [1]:
import pandas as pd

In [2]:
print(pd.__version__)

0.18.0


## Reading Data

You continue to use pandas for XLS files reading. Suppose you have the `rollingsales_AREA.xls` files in the `data` directory.

The relevant row number to extract header is the row no.5 (note: row spreadsheet begins to 1).

In [3]:
def read_data(fname):
    """Read data from an Excel file
    """
    return pd.read_excel(fname, header=4)

In [4]:
brooklyn = read_data("./data/rollingsales_brooklyn.xls")

In [5]:
brooklyn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23373 entries, 0 to 23372
Data columns (total 21 columns):
BOROUGH                           23373 non-null int64
NEIGHBORHOOD                      23373 non-null object
BUILDING CLASS CATEGORY           23373 non-null object
TAX CLASS AT PRESENT              23373 non-null object
BLOCK                             23373 non-null int64
LOT                               23373 non-null int64
EASE-MENT                         23373 non-null object
BUILDING CLASS AT PRESENT         23373 non-null object
ADDRESS                           23373 non-null object
APART
MENT
NUMBER                 23373 non-null object
ZIP CODE                          23373 non-null int64
RESIDENTIAL UNITS                 23373 non-null int64
COMMERCIAL UNITS                  23373 non-null int64
TOTAL UNITS                       23373 non-null int64
LAND SQUARE FEET                  23373 non-null int64
GROSS SQUARE FEET                 23373 non-null int64
YEAR

21 columns and several different data

## Load and Clean up

Quoting: *First challenge: load in and clean up the data. Next, conduct
exploratory data analysis in order to find out where there are
outliers or missing values, decide how you will treat them, make
sure the dates are formatted correctly, make sure values you
think are numerical are being treated as such, etc.*

Rename some column names (there some \n)

In [6]:
def clean_columns(df):
    """Clean some columns name
    """
    df.columns = [x.replace('\n', ' ').lower() for x in df.columns]
    return df.rename_axis({'apart ment number': 'apartment number'}, axis=1)
    
def missing_string(df, colnames):
    """Strip content and fill empty string with NaN
    """
    df = df.copy()
    for col in colnames:
        df[col] = df[col].str.strip().apply(lambda x: 'NaN' if not x else x)
    return df

Column `apart ment number` sounds strange and this is an object (i.e. a string). Rename it and check if it can be integers.

**update** the clean columns function.

In [7]:
brooklyn = clean_columns(brooklyn)

In [9]:
brooklyn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23373 entries, 0 to 23372
Data columns (total 21 columns):
borough                           23373 non-null int64
neighborhood                      23373 non-null object
building class category           23373 non-null object
tax class at present              23373 non-null object
block                             23373 non-null int64
lot                               23373 non-null int64
ease-ment                         23373 non-null object
building class at present         23373 non-null object
address                           23373 non-null object
apartment number                  23373 non-null object
zip code                          23373 non-null int64
residential units                 23373 non-null int64
commercial units                  23373 non-null int64
total units                       23373 non-null int64
land square feet                  23373 non-null int64
gross square feet                 23373 non-null int64
year

In [20]:
brooklyn['apartment number'].sample(10)

4824                 
4223                 
21382    4C          
13851                
18186                
7375                 
99                   
7579                 
6079                 
6935                 
Name: apartment number, dtype: object

OK, sometimes, you can have two digits or one digits and letters. We can normalize that, i.e. discard some trailing whitespaces. And replace every empty string by 'NaN'.

In [10]:
brooklyn = missing_string(brooklyn, ['apartment number'])

Let's see if you have other string field with the same issues

In [12]:
brooklyn['apartment number'].sample(10)

2024     NaN
8121     NaN
20948    NaN
13587    NaN
19512    NaN
19236     8B
4016     NaN
20470    NaN
10353    NaN
1305     NaN
Name: apartment number, dtype: object

In [13]:
brooklyn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23373 entries, 0 to 23372
Data columns (total 21 columns):
borough                           23373 non-null int64
neighborhood                      23373 non-null object
building class category           23373 non-null object
tax class at present              23373 non-null object
block                             23373 non-null int64
lot                               23373 non-null int64
ease-ment                         23373 non-null object
building class at present         23373 non-null object
address                           23373 non-null object
apartment number                  21744 non-null object
zip code                          23373 non-null int64
residential units                 23373 non-null int64
commercial units                  23373 non-null int64
total units                       23373 non-null int64
land square feet                  23373 non-null int64
gross square feet                 23373 non-null int64
year

WARNING: a `0` value can occur in the "Year built" column.

### Type of Columns

Are integer and string columns qualitative or sequential data?

* Districts or zipcodes could be qualitative data
* Years or prices could be sequential data (i.e. you can sort them)

In [20]:
def number_unique_values(data):
    """Count the number of unique values for each column
    
    Return a DataFrame with the name of the columns, the type and the number of unique values
    """
    res = pd.DataFrame([(name, data[name].dtype.name, data[name].nunique()) for name in data],
                      columns=['name', 'type', 'number'])
    return res.sort_values(by=['type', 'number'])

In [21]:
nuniq = number_unique_values(brooklyn)

In [22]:
nuniq

,name,type,number
20,sale date,datetime64[ns],373
0,borough,int64,1
17,tax class at time of sale,int64,4
12,commercial units,int64,27
10,zip code,int64,41
11,residential units,int64,91
13,total units,int64,96
16,year built,int64,145
5,lot,int64,1134
14,land square feet,int64,2552


**Note** that *most of the time*, `object` type means `string`.

When you have a small number of unique values in comparison with the number of rows, it could be qualitative data.

For a predictive model, the columns with a unique value are irrelevant.

### Crazy Values

Data are always messed up. Check if some integer/float columns contain some crazy values, e.g. -999 for a positive float or 0 for years...

In [25]:
def range_columns(data):
    """Min/max values for each integer columns
    """
    intcols = data.dtypes[data.dtypes == 'int64'].index.tolist()
    res = data[intcols].describe().T
    return res[["min", "50%", "max", "mean"]]

In [28]:
range_columns(brooklyn)

,min,50%,max,mean
borough,3.0,3.0,3.0,3.000000
block,20.0,3839.0,8955.0,3984.270312
lot,1.0,48.0,9039.0,305.441064
zip code,0.0,11218.0,11416.0,11211.478929
residential units,0.0,1.0,509.0,2.155864
commercial units,0.0,0.0,222.0,0.197322
total units,0.0,1.0,509.0,2.369700
land square feet,0.0,1850.0,7446955.0,3558.953408
gross square feet,0.0,1760.0,2548000.0,3005.279896
year built,0.0,1925.0,2013.0,1681.096436


zip code, sale price or year equal to `0` is quite strange!